# Train Deep Learning Model

This notebook consists of two tasks. For both tasks, you must provide your assigned dataset name in list format, just as you did in the previous exercise.

When you train a model, it will print the following evaluation metrics:
- `Training loss`
- `Training time`
- `Mean Absolute Error (MAE)`
- `Root Mean Squared Error (RMSE)`
- `R² score`

Below is an overview of the three metrics:

### Evaluation Metrics and Their Desired Trends

| Metric             | Definition                                                                                  | Desired Trend                                  |
|--------------------|----------------------------------------------------------------------------------------------|------------------------------------------------|
| MAE (Mean Absolute Error) | Measures the average absolute difference between predicted and actual values.           | ↓ Decrease — Lower MAE indicates better average accuracy. |
| RMSE (Root Mean Squared Error)  | Measures the average of squared differences; penalizes larger errors more heavily.      | ↓ Decrease — Lower RMSE indicates fewer large errors.       |
| R² (Coefficient of Determination) | Indicates the proportion of variance explained by the model (maximum = 1.0).        | ↑ Increase — Higher R² means better model fit.             |


Please familiarize yourself with the metrics above, as they will be important for completing the analysis in the next tutorial.

After model evaluation is complete, a folder will be created in your current directory using the following structure:
`dataset_name → model_name → activation_function → optimizer_name → epoch_num`

Three JSON files will be generated within this directory structure: 
1) `predictions.json`
2) `evaluation.json`
3) `train_loss.json`

The predictions.json file stores the input data points used for forecasting, along with the predicted values and corresponding target values. The evaluation.json file contains the evaluation metrics, while the train_loss.json file records the training loss and model training time.

## Task 1: Train Predefined Models

This task involves training four predefined model architectures using the dataset you have been assigned. These models include:

* `NN`
* `RNN` (Recurrent Neural Network)
* `LSTM` (Long Short-Term Memory)
* `GRU` (Gated Recurrent Unit)

Note: `LSTM` and `GRU` are specialized types of `RNNs` designed to handle sequence data more effectively.

You are not required to train all combinations — there are 125 or more available — but you should train at least 30 different combinations. Make sure these include at least four combinations for each model architecture (`NN`, `RNN`, `LSTM`, and `GRU`) to ensure broad coverage.

Example Model Combinations
```
| Model | Activation Function | Optimizer | Epochs |
|-------|---------------------|-----------|--------|
| NN    | ReLU                | AdamW     | 10     |
| RNN   | Tanh                | SGD       | 5      |
| LSTM  | GELU                | Adam      | 10     |
| GRU   | Leaky ReLU          | AdamW     | 15     |
```

The code is configured to run over 125 model combinations, which may take significant time to complete.

To speed up development and avoid long runtimes, you can reduce the number of combinations by limiting the range of options.

Shrinked Option Set (Example 1)
```
model_classes = ["NN", "RNN"]
activations = ["relu", "tanh"]
optimizers = ["adam"]
epoch_options = [5, 10]
```

Shrinked Option Set (Example 2 — Quick Debug)
```
model_classes = ["NN"]
activations = ["relu"]
optimizers = ["adam"]
epoch_options = [5]
```

Note: Do not attempt to train for more than 30 epochs per combination, as this may lead to long execution times and unnecessary resource usage.

## Task 2: Build and Train Your Custom Model

This task focuses on building and training your own custom model, referred to as `MyNN`.

In this task, you will design your own model architecture and evaluate its performance by training it under different settings. You should train at least `10` different combinations using `MyNN`.

Example Combinations with `MyNN`
```
| Model | Activation Function | Optimizer | Epochs |
|-------|---------------------|-----------|--------|
| MyNN  | ReLU                | AdamW     | 10     |
| MyNN  | Tanh                | SGD       | 5      |
| MyNN  | GELU                | Adam      | 10     |
| MyNN  | Leaky ReLU          | AdamW     | 15     |
```

In [ ]:
# ------------------- #
# --- Do Not Edit --- #
# ------------------- #

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from buildings_bench import load_torch_dataset
from buildings_bench.models import model_factory

import tomli
from pathlib import Path
import os 
import time
import json
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


class DataHandler:
    def __init__(self, batch_size=32):
        self.batch_size = batch_size

    def load_dataset(self, dataset_name, scaler_transform):
        from buildings_bench import load_torch_dataset
        return list(load_torch_dataset(
            dataset_name,
            apply_scaler_transform=scaler_transform,
            scaler_transform_path=Path(os.environ["TRANSFORM_PATH"])
        ))

    def create_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

class TimeSeriesSinusoidalPeriodicEmbedding(nn.Module):
    def __init__(self, embedding_dim: int):
        super().__init__()
        self.linear = nn.Linear(2, embedding_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.cat([torch.sin(torch.pi * x), torch.cos(torch.pi * x)], dim=2)
        return self.linear(x)

class Model(nn.Module):
    DEFAULT_CONTEXT_LEN = 168
    DEFAULT_PRED_LEN = 24

    def __init__(self, activation):
        super().__init__()
        self.context_len = self.DEFAULT_CONTEXT_LEN
        self.pred_len = self.DEFAULT_PRED_LEN
        self.activation = self._get_activation(activation)
        self.embeddings = self._create_embeddings()

    def _create_embeddings(self):
        return nn.ModuleDict({
            'power': nn.Linear(1, 64),
            'building': nn.Embedding(2, 32),
            'lat': nn.Linear(1, 32),
            'lon': nn.Linear(1, 32), 
            'day_of_year': TimeSeriesSinusoidalPeriodicEmbedding(32),
            'day_of_week': TimeSeriesSinusoidalPeriodicEmbedding(32),
            'hour_of_day': TimeSeriesSinusoidalPeriodicEmbedding(32)
        })

    def _get_activation(self, name):
        return {
            "relu": nn.ReLU(),
            "tanh": nn.Tanh(),
            "gelu": nn.GELU(),
            "leaky_relu": nn.LeakyReLU()
        }.get(name.lower(), nn.ReLU())

    def _data_pre_process(self, x):
        lat = self.embeddings['lat'](x['latitude'])
        lon = self.embeddings['lon'](x['longitude'])
        btype = self.embeddings['building'](x['building_type'].squeeze(-1))
        load = self.embeddings['power'](x['load'])
        day_of_year = self.embeddings['day_of_year'](x['day_of_year'])            
        day_of_week = self.embeddings['day_of_week'](x['day_of_week'])            
        hour_of_day = self.embeddings['hour_of_day'](x['hour_of_day']) 
        return torch.cat([lat, lon, btype, day_of_year, day_of_week, hour_of_day, load], dim=2)

class NN(Model):
    def __init__(self, activation):
        super().__init__(activation)
        self.model = self._build_model()

    def _build_model(self):
        input_dim = self.context_len * 256
        return nn.Sequential(
            nn.Linear(input_dim, 128), 
            self.activation,
            nn.Linear(128, self.pred_len)
        )

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        x_flat = ts_embed[:, :self.context_len, :].reshape(x['load'].shape[0], -1)
        return self.model(x_flat).unsqueeze(-1)


class RNN(Model):
    def __init__(self, activation="relu"):
        super().__init__(activation)
        self.rnn1, self.rnn2, self.output_layer = self._build_model()

    def _build_model(self):
        rnn1 = nn.RNN(256, 128, batch_first=True)
        rnn2 = nn.RNN(128, 128, batch_first=True)
        output_layer = nn.Linear(128, self.pred_len)
        return rnn1, rnn2, output_layer

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        out1, _ = self.rnn1(ts_embed)
        out2, _ = self.rnn2(out1)
        last_hidden = self.activation(out2[:, -1, :])
        return self.output_layer(last_hidden).unsqueeze(-1)

class LSTM(Model):
    def __init__(self, activation="relu"):
        super().__init__(activation)
        self.lstm1, self.lstm2, self.output_layer = self._build_model()

    def _build_model(self):
        lstm1 = nn.LSTM(256, 128, batch_first=True)
        lstm2 = nn.LSTM(128, 128, batch_first=True)
        output_layer = nn.Linear(128, self.pred_len)
        return lstm1, lstm2, output_layer

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        out1, _ = self.lstm1(ts_embed)
        out2, _ = self.lstm2(out1)
        last_hidden = self.activation(out2[:, -1, :])
        return self.output_layer(last_hidden).unsqueeze(-1)

class GRU(Model):
    def __init__(self, activation="relu"):
        super().__init__(activation)
        self.gru1, self.gru2, self.output_layer = self._build_model()

    def _build_model(self):
        gru1 = nn.GRU(256, 128, batch_first=True)
        gru2 = nn.GRU(128, 128, batch_first=True)
        output_layer = nn.Linear(128, self.pred_len)
        return gru1, gru2, output_layer

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        out1, _ = self.gru1(ts_embed)
        out2, _ = self.gru2(out1)
        last_hidden = self.activation(out2[:, -1, :])
        return self.output_layer(last_hidden).unsqueeze(-1)

class Trainer:
    def __init__(self, model_name, device, scaler_transform, dataset_name, epochs, train_buildings, test_buildings, activation='relu', optimizer_name='adam', lr=1e-3):
        self.model_name = model_name
        self.device = device
        self.scaler_transform = scaler_transform
        self.dataset_name = dataset_name
        self.epochs = epochs
        self.train_buildings = train_buildings
        self.test_buildings = test_buildings
        self.activation = activation
        self.optimizer_name = optimizer_name
        self.lr = lr
        self.model = self._load_model()
        self.optimizer = self._get_optimizer()
        self.loss_fn = nn.MSELoss()
        self.handler = DataHandler(batch_size=32)
        self.path = os.path.join(os.getcwd(), self.dataset_name, self.model_name, self.activation, self.optimizer_name, f'epochs-{self.epochs}')
        os.makedirs(self.path, exist_ok=True)

    def _load_model(self):
        model_map = {
            'NN': NN,
            'RNN': RNN,
            'LSTM': LSTM,
            'GRU': GRU, 
            'MyNN': MyNN
        }
        return model_map[self.model_name](activation=self.activation).to(self.device)

    def _get_optimizer(self):
        opt_map = {
            'adam': torch.optim.Adam,
            'sgd': torch.optim.SGD,
            'adamw': torch.optim.AdamW
        }
        optimizer_cls = opt_map.get(self.optimizer_name.lower(), torch.optim.Adam)
        return optimizer_cls(self.model.parameters(), lr=self.lr)

    def train(self):
        self.model.train()
        log = []
        start_time = time.time()  # Start timer
        for epoch in range(self.epochs):
            total_loss = 0.0
            for building_id, building_dataset in self.train_buildings:
                dataloader = self.handler.create_dataloader(building_dataset)
                for batch in dataloader:
                    for key, value in batch.items():
                        batch[key] = value.to(self.device)
                    self.optimizer.zero_grad()
                    predictions = self.model(batch)
                    targets = batch['load'][:, self.model.context_len:, 0]
                    loss = self.loss_fn(predictions[:, :, 0], targets)
                    loss.backward()
                    self.optimizer.step()
                    total_loss += loss.item()
            print(f"[{self.model_name}] Epoch {epoch + 1}: Loss = {total_loss:.4f}")
            log.append({"epoch": epoch + 1, "loss": total_loss})
        train_duration = time.time() - start_time  # End timer
        with open(os.path.join(self.path, "train_loss.json"), "w") as f:
             json.dump({"train_loss": log, "train_duration": train_duration}, f, indent=2)
        return train_duration

    def evaluate(self):
        self.model.eval()
        results = {}
        mae_total = 0.0
        rmse_total = 0.0
        r2_total = 0.0
        count = 0
        for building_id, building_dataset in self.test_buildings:
            inverse_transform = building_dataset.datasets[0].load_transform.undo_transform
            dataloader = self.handler.create_dataloader(building_dataset)
            
            target_list = []
            prediction_list = []
            load_list = []
            
            with torch.no_grad():
                for batch in dataloader:
                    for key, value in batch.items():
                        batch[key] = value.to(self.device)

                    
                    predictions = self.model(batch)
                    targets = batch['load'][:, self.model.context_len:]
                    loads = batch['load'][:, :self.model.context_len]
                    
                    targets = inverse_transform(targets)
                    predictions = inverse_transform(predictions)
                    loads = inverse_transform(loads)
                    
                    prediction_list.append(predictions.detach().cpu())
                    target_list.append(targets.detach().cpu())
                    load_list.append(loads.detach().cpu())
            
            predictions_all = torch.cat(prediction_list)
            targets_all = torch.cat(target_list)
            load_all = torch.cat(load_list)
            
            mae = torch.abs(predictions_all - targets_all).mean().item()
            rmse = torch.sqrt(((predictions_all - targets_all) ** 2).mean()).item()
            r2 = 1 - (((predictions_all - targets_all) ** 2).sum() / ((targets_all - targets_all.mean()) ** 2).sum()).item()
            mae_total += mae
            rmse_total += rmse
            r2_total += r2
            count += 1
            results[building_id] = {
                "load": load_all.tolist(),
                "predictions": predictions_all.tolist(),
                "targets": targets_all.tolist()
            }
        with open(os.path.join(self.path, "predictions.json"), "w") as f:
            json.dump(results, f, indent=2)
        eval_metrics = {
            "mae": mae_total / count,
            "rmse": rmse_total / count,
            "r2": r2_total / count}
        with open(os.path.join(self.path, "evaluate_model.json"), "w") as f:
            json.dump(eval_metrics, f, indent=2)
        return results, eval_metrics["mae"], eval_metrics["rmse"], eval_metrics["r2"]

# ------------------- #
# --- Do Not Edit --- #
# ------------------- #

## Task 1

In [22]:
# ------------------- #
# ------ Edit ------- #
# ------------------- #
# dataset_names = ["ideal"] # TODO: Provide the dataset name as a list using the format [dataset_name] instead of passing it as a variable
# # TODO: Explore at least 30 different combinations using all four models
# model_classes = ["NN", "RNN", "LSTM", "GRU"]
# activations = ["relu", "tanh", "leaky_relu", "gelu"]
# optimizers = ["adam", "sgd", "adamw"]
# epoch_options = [5, 10, 15]
dataset_names = ["electricity"] # TODO: Provide the dataset name as a list using the format [dataset_name]
# TODO: Explore at least 30 different combinations using all four models
model_classes = ["RNN"]
activations = ["leaky_relu", "tanh"]
optimizers = ["adamw", "sgd"]
epoch_options = [2]
# ------------------- #
# ------ Edit ------- #
# ------------------- #
# ------------------- #
# --- Do Not Edit --- #
# ------------------- #
class MyNN(Model):
     def __init__(self):
         pass
os.environ["REPO_PATH"] = "/global/cfs/cdirs/m4388/Project4/BuildingsBench"
os.environ["BUILDINGS_BENCH"] = "/global/cfs/cdirs/m4388/Project4/Dataset"
os.environ["TRANSFORM_PATH"] = "/global/cfs/cdirs/m4388/Project4/Dataset/metadata/transforms"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
for dataset_name in dataset_names:
    print(f"\n=== Dataset: {dataset_name} ===")
    handler = DataHandler(batch_size=32)
    all_buildings = handler.load_dataset(dataset_name, scaler_transform="boxcox")
    train_buildings = all_buildings[:int(0.8 * len(all_buildings))]
    test_buildings = all_buildings[int(0.8 * len(all_buildings)):]
    for model_class in model_classes:
        for activation in activations:
            for optimizer_name in optimizers:
                for epochs in epoch_options:
                    print(f"\n--- Training {model_class} | Activation: {activation} | Optimizer: {optimizer_name} | Epochs: {epochs} ---")
                    trainer = Trainer(
                        model_name=model_class,
                        device=device,
                        dataset_name=dataset_name,
                        epochs=epochs,
                        train_buildings=train_buildings,
                        test_buildings=test_buildings,
                        scaler_transform="boxcox",
                        activation=activation,
                        optimizer_name=optimizer_name,
                        lr=1e-3)
                    train_duration = trainer.train()
                    results, mae, rmse, r2 = trainer.evaluate()
                    print(f"[{model_class}] MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
                    print(f"Training Time: {train_duration:.2f} seconds")
# ------------------- #
# --- Do Not Edit --- #
# -------------------#








=== Dataset: electricity ===


/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. 


--- Training RNN | Activation: leaky_relu | Optimizer: adamw | Epochs: 2 ---
[RNN] MAE: 1540.5220, RMSE: 1774.9586, R²: -254.7830
Training Time: 1009.32 seconds

--- Training RNN | Activation: leaky_relu | Optimizer: sgd | Epochs: 2 ---
[RNN] Epoch 1: Loss = 1314.5254
[RNN] Epoch 2: Loss = 394.8243
[RNN] MAE: 902.7325, RMSE: 1117.5636, R²: 0.3583
Training Time: 1036.64 seconds

--- Training RNN | Activation: tanh | Optimizer: adamw | Epochs: 2 ---
[RNN] Epoch 1: Loss = 1198.0879
[RNN] Epoch 2: Loss = 1234.5671
[RNN] MAE: 1697.8887, RMSE: 1939.5182, R²: -369.1338
Training Time: 1065.86 seconds

--- Training RNN | Activation: tanh | Optimizer: sgd | Epochs: 2 ---
[RNN] Epoch 1: Loss = 1204.2811
[RNN] Epoch 2: Loss = 307.5832
[RNN] MAE: 874.2019, RMSE: 1091.2731, R²: 0.3944
Training Time: 1025.81 seconds


## Task 2

In [21]:
class MyNN(Model):
    def __init__(self, activation):
        super().__init__(activation)
        self.model = self._build_model()

    def _get_activation(self, activation):
        # Map the string to a torch.nn activation; default to ReLU if unknown
        name = (activation or "").lower()
        if name == "relu":
            return nn.ReLU()
        if name == "tanh":
            return nn.Tanh()
        if name == "leaky_relu":
            return nn.LeakyReLU()
        if name == "gelu":
            return nn.GELU()
        return nn.ReLU()

    def _build_model(self):
        input_dim = self.context_len * 256
        act = self.activation  # use the already created activation from parent init
        return nn.Sequential(
            # TODO: Create an input layer using input_dim as the dimension parameter
            nn.Linear(input_dim, 512),

            # TODO: Add at least three hidden layers
            act,
            nn.Linear(512, 256),
            act,
            nn.Linear(256, 128),
            act,

            # TODO: Create an output layer using self.pred_len as the dimension parameter
            nn.Linear(128, self.pred_len)
        )

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        x_flat = ts_embed[:, :self.context_len, :].reshape(x['load'].shape[0], -1)
        return self.model(x_flat).unsqueeze(-1)


# TODO: Provide the dataset name as a list using the format [dataset_name]
dataset_names = ["electricity"]

# TODO: Explore at least 10 different combinations using MyNN
model_classes = ["MyNN"]
activations = ["relu", "tanh", "leaky_relu", "gelu"]
optimizers = ["adam", "sgd", "adamw"]
epoch_options = [1]


# ------------------- #
# ------ Edit ------- #
# ------------------- #

# ------------------- #
# --- Do Not Edit --- #
# ------------------- #

os.environ["REPO_PATH"] = "/global/cfs/cdirs/m4388/Project4/BuildingsBench"
os.environ["BUILDINGS_BENCH"] = "/global/cfs/cdirs/m4388/Project4/Dataset"
os.environ["TRANSFORM_PATH"] = "/global/cfs/cdirs/m4388/Project4/Dataset/metadata/transforms"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

for dataset_name in dataset_names:
    print(f"\n=== Dataset: {dataset_name} ===")
    handler = DataHandler(batch_size=32)
    all_buildings = handler.load_dataset(dataset_name, scaler_transform="boxcox")
    train_buildings = all_buildings[:int(0.8 * len(all_buildings))]
    test_buildings = all_buildings[int(0.8 * len(all_buildings)):]
    for model_class in model_classes:
        for activation in activations:
            for optimizer_name in optimizers:
                for epochs in epoch_options:
                    print(f"\n--- Training {model_class} | Activation: {activation} | Optimizer: {optimizer_name} | Epochs: {epochs} ---")
                    trainer = Trainer(
                        model_name=model_class,
                        device=device,
                        dataset_name=dataset_name,
                        epochs=epochs,
                        train_buildings=train_buildings,
                        test_buildings=test_buildings,
                        scaler_transform="boxcox",
                        activation=activation,
                        optimizer_name=optimizer_name,
                        lr=1e-3)
                    train_duration = trainer.train()
                    results, mae, rmse, r2 = trainer.evaluate()
                    print(f"[{model_class}] MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
                    print(f"Training Time: {train_duration:.2f} seconds")

# ------------------- #
# --- Do Not Edit --- #
# ------------------- #



=== Dataset: electricity ===


/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. 


--- Training MyNN | Activation: relu | Optimizer: adam | Epochs: 1 ---


KeyboardInterrupt: 

## Next Step

`/BuildingsBenchTutorial/Tutorials/Final-Project-Modules/Select-Model.ipynb`

In [4]:
model.save("model_checkpoint.h5")

NameError: name 'model' is not defined

In [5]:
whi

NameError: name 'whi' is not defined

In [6]:
who

DataHandler	 DataLoader	 GRU	 LSTM	 Model	 MyNN	 NN	 Path	 RNN	 
TimeSeriesSinusoidalPeriodicEmbedding	 Trainer	 activation	 activations	 all_buildings	 dataset_name	 dataset_names	 device	 epoch_options	 
epochs	 handler	 json	 load_torch_dataset	 mae	 mean_absolute_error	 mean_squared_error	 model_class	 model_classes	 
model_factory	 nn	 np	 optimizer_name	 optimizers	 os	 r2	 r2_score	 results	 
rmse	 test_buildings	 time	 tomli	 torch	 train_buildings	 train_duration	 trainer	 


In [ ]:
# this tests all the models and will figure out the best combo
# ------------------- #
# ------ Edit ------- #
# ------------------- #

# Your improved MyNN for fast one-epoch learning
class MyNN(Model):
    def __init__(self, activation):
        super().__init__(activation)
        self.act = self._get_activation(activation)
        self.model = self._build_model()
        self._init_weights()

    def _build_model(self):
        input_dim = self.context_len * 256
        h1 = max(128, min(1024, input_dim // 2))
        h2 = max(64, min(512,  h1 // 2))
        h3 = max(32, min(256,  h2 // 2))
        p = 0.15
        return nn.Sequential(
            nn.Linear(input_dim, h1), nn.BatchNorm1d(h1), self.act, nn.Dropout(p),
            nn.Linear(h1, h2),        nn.BatchNorm1d(h2), self.act, nn.Dropout(p),
            nn.Linear(h2, h3),        nn.BatchNorm1d(h3), self.act, nn.Dropout(p),
            nn.Linear(h3, self.pred_len)
        )

    def _init_weights(self):
        for m in self.model.modules():
            if isinstance(m, nn.Linear):
                if isinstance(self.act, (nn.ReLU, nn.LeakyReLU, nn.GELU)):
                    nn.init.kaiming_uniform_(m.weight, nonlinearity="relu")
                elif isinstance(self.act, nn.Tanh):
                    nn.init.xavier_uniform_(m.weight)
                else:
                    nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        x_flat = ts_embed[:, :self.context_len, :].reshape(x['load'].shape[0], -1)
        return self.model(x_flat).unsqueeze(-1)


# Dataset and sweep settings
dataset_names = ["electricity"]   # REQUIRED
model_classes = ["MyNN"]
activations = ["gelu", "relu", "leaky_relu", "tanh"]  # 4 activations
optimizers = ["adamw", "adam", "sgd"]                # 3 optimizers
epoch_options = [1]                                  # fast training

# ------------------- #
# --- Run Loop ---    #
# ------------------- #

os.environ["REPO_PATH"] = "/global/cfs/cdirs/m4388/Project4/BuildingsBench"
os.environ["BUILDINGS_BENCH"] = "/global/cfs/cdirs/m4388/Project4/Dataset"
os.environ["TRANSFORM_PATH"] = "/global/cfs/cdirs/m4388/Project4/Dataset/metadata/transforms"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

for dataset_name in dataset_names:
    print(f"\n=== Dataset: {dataset_name} ===")
    handler = DataHandler(batch_size=32)
    all_buildings = handler.load_dataset(dataset_name, scaler_transform="boxcox")
    train_buildings = all_buildings[:int(0.8 * len(all_buildings))]
    test_buildings = all_buildings[int(0.8 * len(all_buildings)):]
    for model_class in model_classes:
        for activation in activations:
            for optimizer_name in optimizers:
                for epochs in epoch_options:
                    print(f"\n--- Training {model_class} | Activation: {activation} | Optimizer: {optimizer_name} | Epochs: {epochs} ---")
                    trainer = Trainer(
                        model_name=model_class,
                        device=device,
                        dataset_name=dataset_name,
                        epochs=epochs,
                        train_buildings=train_buildings,
                        test_buildings=test_buildings,
                        scaler_transform="boxcox",
                        activation=activation,
                        optimizer_name=optimizer_name,
                        lr=1e-3)
                    train_duration = trainer.train()
                    results, mae, rmse, r2 = trainer.evaluate()
                    print(f"[{model_class}] MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
                    print(f"Training Time: {train_duration:.2f} seconds")



=== Dataset: electricity ===


/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. 


--- Training MyNN | Activation: gelu | Optimizer: adamw | Epochs: 1 ---
[MyNN] Epoch 1: Loss = 2210.1885
[MyNN] MAE: 1062.7734, RMSE: 1347.5334, R²: -14.3558
Training Time: 465.28 seconds

--- Training MyNN | Activation: gelu | Optimizer: adam | Epochs: 1 ---
[MyNN] Epoch 1: Loss = 2158.6339
[MyNN] MAE: 6985.6502, RMSE: 79496.0821, R²: -148059679.7577
Training Time: 467.71 seconds

--- Training MyNN | Activation: gelu | Optimizer: sgd | Epochs: 1 ---
[MyNN] Epoch 1: Loss = 5479.2613


/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:3174: RuntimeWarning: overflow encountered in power
  x_inv = (x * lmbda + 1) ** (1 / lmbda)


[MyNN] MAE: nan, RMSE: nan, R²: nan
Training Time: 459.68 seconds

--- Training MyNN | Activation: relu | Optimizer: adamw | Epochs: 1 ---


In [12]:
# === Fastest single-combo run: MyNN + GELU + AdamW, 1 epoch, electricity ===

# Strong MyNN tuned for 1-epoch learning
class MyNN(Model):
    def __init__(self, activation):
        super().__init__(activation)
        self.act = self._get_activation(activation)
        self.model = self._build_model()
        self._init_weights()

    def _build_model(self):
        input_dim = self.context_len * 256
        h1 = max(128, min(1024, input_dim // 2))
        h2 = max(64,  min(512,  h1 // 2))
        h3 = max(32,  min(256,  h2 // 2))
        p = 0.15
        return nn.Sequential(
            nn.Linear(input_dim, h1), nn.BatchNorm1d(h1), self.act, nn.Dropout(p),
            nn.Linear(h1, h2),        nn.BatchNorm1d(h2), self.act, nn.Dropout(p),
            nn.Linear(h2, h3),        nn.BatchNorm1d(h3), self.act, nn.Dropout(p),
            nn.Linear(h3, self.pred_len)
        )

    def _init_weights(self):
        for m in self.model.modules():
            if isinstance(m, nn.Linear):
                # Kaiming for ReLU-like (GELU included), Xavier for tanh fallback
                if isinstance(self.act, (nn.ReLU, nn.LeakyReLU, nn.GELU)):
                    nn.init.kaiming_uniform_(m.weight, nonlinearity="relu")
                elif isinstance(self.act, nn.Tanh):
                    nn.init.xavier_uniform_(m.weight)
                else:
                    nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        x_flat = ts_embed[:, :self.context_len, :].reshape(x['load'].shape[0], -1)
        return self.model(x_flat).unsqueeze(-1)


# --- Minimal config: 1 combo, 1 epoch ---
dataset_names = ["electricity"]
model_classes = ["MyNN"]
activations   = ["gelu"]
optimizers    = ["adamw"]
epoch_options = [1]

# --- Run loop ---
os.environ["REPO_PATH"] = "/global/cfs/cdirs/m4388/Project4/BuildingsBench"
os.environ["BUILDINGS_BENCH"] = "/global/cfs/cdirs/m4388/Project4/Dataset"
os.environ["TRANSFORM_PATH"] = "/global/cfs/cdirs/m4388/Project4/Dataset/metadata/transforms"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

for dataset_name in dataset_names:
    print(f"\n=== Dataset: {dataset_name} ===")
    handler = DataHandler(batch_size=32)
    all_buildings = handler.load_dataset(dataset_name, scaler_transform="boxcox")
    train_buildings = all_buildings[:int(0.8 * len(all_buildings))]
    test_buildings  = all_buildings[int(0.8 * len(all_buildings)):]
    for model_class in model_classes:
        for activation in activations:
            for optimizer_name in optimizers:
                for epochs in epoch_options:
                    print(f"\n--- Training {model_class} | Activation: {activation} | Optimizer: {optimizer_name} | Epochs: {epochs} ---")
                    trainer = Trainer(
                        model_name=model_class,
                        device=device,
                        dataset_name=dataset_name,
                        epochs=epochs,
                        train_buildings=train_buildings,
                        test_buildings=test_buildings,
                        scaler_transform="boxcox",
                        activation=activation,
                        optimizer_name=optimizer_name,
                        lr=1e-3
                    )
                    train_duration = trainer.train()
                    results, mae, rmse, r2 = trainer.evaluate()
                    print(f"[{model_class}] MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
                    print(f"Training Time: {train_duration:.2f} seconds")



=== Dataset: electricity ===


/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. 


--- Training MyNN | Activation: gelu | Optimizer: adamw | Epochs: 1 ---


OutOfMemoryError: CUDA out of memory. Tried to allocate 168.00 MiB. GPU 0 has a total capacity of 39.38 GiB of which 87.12 MiB is free. Process 1555621 has 18.58 GiB memory in use. Process 408374 has 416.00 MiB memory in use. Process 624542 has 416.00 MiB memory in use. Process 2183182 has 18.49 GiB memory in use. Including non-PyTorch memory, this process has 1.37 GiB memory in use. Of the allocated memory 878.89 MiB is allocated by PyTorch, and 7.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
# --- Low-memory MyNN ---
class MyNN(Model):
    def __init__(self, activation):
        super().__init__(activation)
        self.act = self._get_activation(activation)
        self.model = self._build_model()
        self._init_weights()

    def _build_model(self):
        # Smaller hidden sizes to save memory
        h1, h2, h3 = 256, 128, 64
        p = 0.1
        input_dim = self.context_len * 256
        return nn.Sequential(
            nn.Linear(input_dim, h1), nn.BatchNorm1d(h1), self.act, nn.Dropout(p),
            nn.Linear(h1, h2),        nn.BatchNorm1d(h2), self.act, nn.Dropout(p),
            nn.Linear(h2, h3),        nn.BatchNorm1d(h3), self.act, nn.Dropout(p),
            nn.Linear(h3, self.pred_len)
        )

    def _init_weights(self):
        for m in self.model.modules():
            if isinstance(m, nn.Linear):
                if isinstance(self.act, (nn.ReLU, nn.LeakyReLU, nn.GELU)):
                    nn.init.kaiming_uniform_(m.weight, nonlinearity="relu")
                elif isinstance(self.act, nn.Tanh):
                    nn.init.xavier_uniform_(m.weight)
                else:
                    nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        ts_embed = self._data_pre_process(x)
        x_flat = ts_embed[:, :self.context_len, :].reshape(x['load'].shape[0], -1)
        return self.model(x_flat).unsqueeze(-1)


# --- Config ---
dataset_names = ["electricity"]
model_classes = ["MyNN"]
activations   = ["gelu"]
optimizers    = ["adamw"]
epoch_options = [1]

# --- Run ---
os.environ["REPO_PATH"] = "/global/cfs/cdirs/m4388/Project4/BuildingsBench"
os.environ["BUILDINGS_BENCH"] = "/global/cfs/cdirs/m4388/Project4/Dataset"
os.environ["TRANSFORM_PATH"] = "/global/cfs/cdirs/m4388/Project4/Dataset/metadata/transforms"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

for dataset_name in dataset_names:
    print(f"\n=== Dataset: {dataset_name} ===")
    handler = DataHandler(batch_size=16)  # smaller batch size
    all_buildings = handler.load_dataset(dataset_name, scaler_transform="boxcox")
    train_buildings = all_buildings[:int(0.8 * len(all_buildings))]
    test_buildings  = all_buildings[int(0.8 * len(all_buildings)):]
    for model_class in model_classes:
        for activation in activations:
            for optimizer_name in optimizers:
                for epochs in epoch_options:
                    print(f"\n--- Training {model_class} | Activation: {activation} | Optimizer: {optimizer_name} | Epochs: {epochs} ---")
                    trainer = Trainer(
                        model_name=model_class,
                        device=device,
                        dataset_name=dataset_name,
                        epochs=epochs,
                        train_buildings=train_buildings,
                        test_buildings=test_buildings,
                        scaler_transform="boxcox",
                        activation=activation,
                        optimizer_name=optimizer_name,
                        lr=1e-3
                    )
                    train_duration = trainer.train()
                    results, mae, rmse, r2 = trainer.evaluate()
                    print(f"[{model_class}] MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
                    print(f"Training Time: {train_duration:.2f} seconds")



=== Dataset: electricity ===


/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/global/homes/k/kareem8/.conda/envs/BBenchEnv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. 


--- Training MyNN | Activation: gelu | Optimizer: adamw | Epochs: 1 ---


OutOfMemoryError: CUDA out of memory. Tried to allocate 42.00 MiB. GPU 0 has a total capacity of 39.38 GiB of which 25.12 MiB is free. Process 1555621 has 18.58 GiB memory in use. Process 408374 has 416.00 MiB memory in use. Process 624542 has 416.00 MiB memory in use. Process 2183182 has 18.49 GiB memory in use. Including non-PyTorch memory, this process has 1.43 GiB memory in use. Of the allocated memory 932.83 MiB is allocated by PyTorch, and 15.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)